In [12]:
#Imports e configuração de caminho
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os

# Cria pasta de saída
os.makedirs("data/processed", exist_ok=True)

In [13]:
#Carrega os dados brutos
df_dengue = pd.read_csv("data/raw/dengue.csv")
df_clima  = pd.read_csv("data/raw/clima_inmet.csv")
df_pop    = pd.read_csv("data/raw/pop_density.csv")

print("Dengue:", df_dengue.shape)
print("Clima:", df_clima.shape)
print("População:", df_pop.shape)

Dengue: (812, 30)
Clima: (313, 6)
População: (1, 2)


In [14]:
#Prepara base de dengue
# Extrai ano e semana epidemiológica
df_dengue["data_iniSE"] = pd.to_datetime(df_dengue["data_iniSE"])
df_dengue["year"] = df_dengue["data_iniSE"].dt.isocalendar().year
df_dengue["week"] = df_dengue["data_iniSE"].dt.isocalendar().week

# Agrupa por semana e município
df_dengue = df_dengue.groupby(["Localidade_id", "year", "week"], as_index=False).agg({
    "casos": "sum",
    "Rt": "mean",
    "pop": "mean",
    "tempmed": "mean",
    "umidmed": "mean"
})

In [15]:
#junta com clima e população
df = df_dengue.merge(df_clima, on=["year", "week"], how="left")
df = df.merge(df_pop, left_on="Localidade_id", right_on="municipio_id", how="left")

In [17]:
# Atualiza tipos antes de interpolar
df = df.infer_objects(copy=False)

# Interpolação e preenchimento com métodos atualizados
df = df.infer_objects(copy=False)
# Converte colunas de tipo 'object' para tipos mais específicos
df = df.infer_objects(copy=False)

#Aplica interpolação e preenchimento
df = df.interpolate(method="linear")
df = df.ffill()
df = df.bfill()
df = df.ffill()
df = df.bfill()

C:\Users\valdiney.pedro\AppData\Local\Temp\ipykernel_6740\1246418781.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate(method="linear")
